In [8]:
!pip install pykrx

Defaulting to user installation because normal site-packages is not writeable


In [9]:
from pykrx import stock
from pykrx import bond


In [10]:
import json

def get_market_ohlcv(start_date, end_date, ticker):
    stock_name = stock.get_market_ticker_name(ticker)
    df = stock.get_market_ohlcv(start_date, end_date, ticker)
    df['종목명'] = [stock_name] * len(df)

    return json.dumps(df.to_dict(orient='records'), ensure_ascii=False)

In [11]:
print(get_market_ohlcv("20220720", "20220810", "005930"))

[{"시가": 61800, "고가": 62100, "저가": 60500, "종가": 60500, "거래량": 16782238, "등락률": -0.6568144499178982, "종목명": "삼성전자"}, {"시가": 61100, "고가": 61900, "저가": 60700, "종가": 61800, "거래량": 12291374, "등락률": 2.1487603305785123, "종목명": "삼성전자"}, {"시가": 61800, "고가": 62200, "저가": 61200, "종가": 61300, "거래량": 10261310, "등락률": -0.8090614886731391, "종목명": "삼성전자"}, {"시가": 60900, "고가": 61900, "저가": 60800, "종가": 61100, "거래량": 9193681, "등락률": -0.3262642740619902, "종목명": "삼성전자"}, {"시가": 60800, "고가": 61900, "저가": 60800, "종가": 61700, "거래량": 6597211, "등락률": 0.9819967266775778, "종목명": "삼성전자"}, {"시가": 61300, "고가": 61900, "저가": 61200, "종가": 61800, "거래량": 7320997, "등락률": 0.1620745542949757, "종목명": "삼성전자"}, {"시가": 62300, "고가": 62600, "저가": 61600, "종가": 61900, "거래량": 10745302, "등락률": 0.16181229773462785, "종목명": "삼성전자"}, {"시가": 62400, "고가": 62600, "저가": 61300, "종가": 61400, "거래량": 15093120, "등락률": -0.8077544426494345, "종목명": "삼성전자"}, {"시가": 61000, "고가": 61700, "저가": 60300, "종가": 61300, "거래량": 13154816, "등락률": -0.1628664495114

In [12]:
tools = [
    {
        "type": "function",
        "function":
        {
            "name": "get_market_ohlcv",
            "description": "특정 종목에 대해 정해진 기간의 주가 데이터를 돌려줍니다.",
            "parameters": {
                "type": "object",
                "properties": {
                    "start_date": {
                        "type": "string",
                        "description": "YYYYMMDD 형식의 시작 날짜",
                    },
                    "end_date": {
                        "type": "string",
                        "description": "YYYYMMDD 형식의 종료 날짜",
                    },
                    "ticker": {
                        "type": "string",
                        "description": "6자리 숫자 형식의 종목 티커코드",
                    },
                },
                "required": ["start_date", "end_date", "ticker"],
            },
        }
    }
]


In [13]:
import os
import openai

openai.api_key = os.getenv("OPENAI_API_KEY")

messages= [
  {
    "role": "user",
    "content": "LG전자(066570)의 2022년 8월 1일부터 2022년 12월 1일까지의 주가 데이터를 분석해줘."
  }
]

response = openai.chat.completions.create(
  model="gpt-4o-mini",
  messages=messages,
  temperature=0,
  max_tokens=1024,
  tools=tools,
  tool_choice="auto" if tools is not None else None,
  top_p=1,
  frequency_penalty=0,
  presence_penalty=0
)

print(response.choices[0].message)

ChatCompletionMessage(content=None, refusal=None, role='assistant', audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_vNMbw9T3UhXj8I4fUSGHZLG8', function=Function(arguments='{"start_date":"20220801","end_date":"20221201","ticker":"066570"}', name='get_market_ohlcv'), type='function')])


In [14]:
available_functions = {
    "get_market_ohlcv": get_market_ohlcv,
}

for tool_call in response.choices[0].message.tool_calls:
    function_name = tool_call.function.name
    function_to_call = available_functions[function_name]
    function_args = json.loads(tool_call.function.arguments)
    function_response = function_to_call(
        start_date=function_args['start_date'],
        end_date=function_args['end_date'],
        ticker=function_args['ticker']
    )
    messages.append(
        {
            "tool_call_id": tool_call.id,
            "role": "function",
            "name": function_name,
            "content": json.dumps(function_response, ensure_ascii=False)
        }
    )
    
second_response = openai.chat.completions.create(
    model="gpt-4o-mini",
    messages=messages,
)
print(second_response.choices[0].message.content)

LG전자의 2022년 8월 1일부터 12월 1일까지의 주가 데이터를 분석해보면 다음과 같은 주요 포인트가 도출됩니다.

### 1. 가격 변동 추세
- **시가**: 시작가는 94,800원에서 시작했습니다.
- **종가**: 12월 1일 기준 종가는 96,200원으로, 이 기간 동안 주가는 상승세를 보였습니다.
- **최고가와 최저가**: 이 기간의 최고가는 101,000원이었으며, 최저가는 77,900원이었습니다.

### 2. 변동 폭
- 주가는 전체적으로 **상승**세를 타며, 여러 차례의 큰 상승과 하락을 겪었습니다. 특히, 9월 초에는 급격한 상승세가 있었으며, 이는 특정 뉴스나 외부 요인에 의해 촉발된 것일 수 있습니다.

### 3. 거래량 분석
- 거래량이 아주 큰 날이 있었습니다. 예를 들어, 9월 5일에는 2,683,987의 거래량을 기록했습니다. 이는 주가의 큰 변동과 관련이 있을 수 있습니다.
- 대체로 거래량은 상승세와 함께 증가하는 경향을 나타냈으며, 큰 변동이 있었던 날에는 특히 높은 거래량이 나타났습니다.

### 4. 등락률
- 여러 날의 **등락률**을 보면, 일일로는 최대 8.84%의 상승률을 보인 날도 있었고, -4% 이상의 하락률을 보인 날도 여러 차례 있었습니다.
- 전체적으로 변동성이 큰 시기였으며, 일부는 급등락을 보였음을 알 수 있습니다.

### 5. 결론
- LG전자의 주가는 해당 기간 동안 상승세를 유지하였으나, 일시적으로 몇 차례 큰 하락이 있었습니다. 다양한 외부 요인이나 기업의 뉴스에 따라 주가가 큰 변동성을 나타냈으며, 투자자들에게는 기회와 위험이 공존하는 시기였음을 보여줍니다.

이 데이터에 기반하여 더욱 심도 깊은 분석을 원한다면, 특정 일자에 대한 추가 정보나 더 긴 기간의 트렌드를 확인하는 것도 좋습니다.
